In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import time
from collections import namedtuple
import cPickle as pickle
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random

from multiprocessing.dummy import Pool as ThreadPool
import itertools

from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial


ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234
WORD2VEC_SEED = 1234

In [4]:
MIN_WORD_COUNT = 100
NUM_CORES = 8

In [5]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [6]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_MATRIX = "validation_matrix.pkl"
VALIDATION_DICT = "validation_dict.pkl"
TEST_MATRIX = "test_matrix.pkl"
TEST_DICT = "test_dict.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"
TYPE_CLASSIFIER= "{}_classifier.pkl"

In [7]:
SAMPLE_RATIO = 0.15

In [25]:
root_location = "/home/local/shalaby/"
exports_location = root_location + "exported_data/"

doc2vec_model_save_location = os.path.join(root_location, "parameter_search_doc2vec_models_extended_abs_desc_claims_large_sample_chunks", "full")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

#training_file = root_location + "docs_output.json"
training_file = root_location + 'docs_output.json'

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
valid_classes_file = exports_location + "valid_classes.pkl"
valid_subclasses_file = exports_location + "valid_subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
# training_docs_list_file = exports_location + "extended_pv_training_docs_list.pkl"
# validation_docs_list_file = exports_location + "extended_pv_validation_docs_list.pkl"
# test_docs_list_file = exports_location + "extended_pv_test_docs_list.pkl"
training_docs_list_file = exports_location + "extended_pv_training_docs_list_" + str(SAMPLE_RATIO) + ".pkl"
validation_docs_list_file = exports_location + "extended_pv_validation_docs_list_" + str(SAMPLE_RATIO) + ".pkl"
test_docs_list_file = exports_location + "extended_pv_test_docs_list_" + str(SAMPLE_RATIO) + ".pkl"

preprocessed_location = root_location + "preprocessed_data/" + "extended_pv_abs_desc_claims_large_sample_chunks/"

training_preprocessed_files_prefix = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
validation_preprocessed_files_prefix = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

docs_only_preprocessed_file = root_location + "preprocessed_data/" + "extended_pv_docs_only_for_spark"


In [9]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
valid_classes = pickle.load(open(valid_classes_file))
valid_subclasses = pickle.load(open(valid_subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 25.2 s, sys: 3min 11s, total: 3min 36s
Wall time: 3min 36s


In [10]:
len(training_docs_list)

254767

In [11]:
len(validation_docs_list)

60957

In [12]:
len(test_docs_list)

79785

In [189]:
def ensure_disk_location_exists(location):
    if not os.path.exists(location):
        os.makedirs(location)

In [259]:
class ExtendedPVDocsOnlyWriterDocumentBatchIterator(object):
    def __init__(self, filename_prefix, file_to_write_path):
        self.filename_prefix = filename_prefix
        self.file_to_write_path = file_to_write_path
        self.file_to_write = open(file_to_write_path, "a")
        self.curr_doc_index = 0
    def load_new_batch_in_memory(self):
        info("Loading new batch for index: {}".format(self.curr_doc_index))
        try:
            preproc_file  = open(self.filename_prefix + str(self.curr_doc_index), "r")
            info("Finished loading new batch of {} documents".format(self.curr_doc_index))
            return preproc_file
        except:
            return False
    def iterate_over_all_docs(self): 
        preproc_file = self.load_new_batch_in_memory()
        while preproc_file is not False:
            for line in preproc_file:
                try:
                    doc_id, doc_content = line.split(" ", 1)
                    if is_real_doc(doc_id):
                        #print line[:100]
#                         self.file_to_write.write(line + "\n")
                        self.file_to_write.write(line)
                        self.curr_doc_index += 1
#                         if self.curr_doc_index % 100 == 0:
#                             break
                except:
                    continue
            preproc_file = self.load_new_batch_in_memory()
        self.file_to_write.close()
def is_real_doc(doc_id):
    return doc_id.find("_") == -1

## Remove old file contents

In [260]:
dd = open(docs_only_preprocessed_file, "r+")
dd.seek(0)
dd.truncate()
dd.close()

## Iterate to fill it out

In [261]:
training_iterator = ExtendedPVDocsOnlyWriterDocumentBatchIterator(training_preprocessed_files_prefix, 
                                                                  docs_only_preprocessed_file)
training_iterator.iterate_over_all_docs()

2017-03-30 01:44:13,689 : INFO : Loading new batch for index: 0
2017-03-30 01:44:13,691 : INFO : Finished loading new batch of 0 documents
2017-03-30 01:44:15,834 : INFO : Loading new batch for index: 10000
2017-03-30 01:44:15,836 : INFO : Finished loading new batch of 10000 documents
2017-03-30 01:44:18,226 : INFO : Loading new batch for index: 20000
2017-03-30 01:44:18,229 : INFO : Finished loading new batch of 20000 documents
2017-03-30 01:44:20,433 : INFO : Loading new batch for index: 30000
2017-03-30 01:44:20,434 : INFO : Finished loading new batch of 30000 documents
2017-03-30 01:44:23,698 : INFO : Loading new batch for index: 40000
2017-03-30 01:44:23,700 : INFO : Finished loading new batch of 40000 documents
2017-03-30 01:44:26,007 : INFO : Loading new batch for index: 50000
2017-03-30 01:44:26,010 : INFO : Finished loading new batch of 50000 documents
2017-03-30 01:44:28,281 : INFO : Loading new batch for index: 60000
2017-03-30 01:44:28,283 : INFO : Finished loading new batc

In [262]:
validation_iterator = ExtendedPVDocsOnlyWriterDocumentBatchIterator(validation_preprocessed_files_prefix, 
                                                                  docs_only_preprocessed_file)
validation_iterator.iterate_over_all_docs()

2017-03-30 02:05:56,583 : INFO : Loading new batch for index: 0
2017-03-30 02:05:56,584 : INFO : Finished loading new batch of 0 documents
2017-03-30 02:05:58,713 : INFO : Loading new batch for index: 10000
2017-03-30 02:05:58,714 : INFO : Finished loading new batch of 10000 documents
2017-03-30 02:06:14,361 : INFO : Loading new batch for index: 20000
2017-03-30 02:06:14,362 : INFO : Finished loading new batch of 20000 documents
2017-03-30 02:06:22,290 : INFO : Loading new batch for index: 30000
2017-03-30 02:06:22,291 : INFO : Finished loading new batch of 30000 documents
2017-03-30 02:06:47,156 : INFO : Loading new batch for index: 40000
2017-03-30 02:06:47,158 : INFO : Finished loading new batch of 40000 documents
2017-03-30 02:06:50,129 : INFO : Loading new batch for index: 50000
2017-03-30 02:06:50,130 : INFO : Finished loading new batch of 50000 documents
2017-03-30 02:06:55,668 : INFO : Loading new batch for index: 60000
2017-03-30 02:06:55,670 : INFO : Finished loading new batc

In [263]:
test_iterator = ExtendedPVDocsOnlyWriterDocumentBatchIterator(test_preprocessed_files_prefix, 
                                                                  docs_only_preprocessed_file)
test_iterator.iterate_over_all_docs()

2017-03-30 02:47:40,894 : INFO : Loading new batch for index: 0
2017-03-30 02:47:40,897 : INFO : Finished loading new batch of 0 documents
2017-03-30 02:47:43,318 : INFO : Loading new batch for index: 10000
2017-03-30 02:47:43,319 : INFO : Finished loading new batch of 10000 documents
2017-03-30 02:47:46,640 : INFO : Loading new batch for index: 20000
2017-03-30 02:47:46,642 : INFO : Finished loading new batch of 20000 documents
2017-03-30 02:47:49,034 : INFO : Loading new batch for index: 30000
2017-03-30 02:47:49,036 : INFO : Finished loading new batch of 30000 documents
2017-03-30 02:47:51,969 : INFO : Loading new batch for index: 40000
2017-03-30 02:47:51,971 : INFO : Finished loading new batch of 40000 documents
2017-03-30 02:47:54,571 : INFO : Loading new batch for index: 50000
2017-03-30 02:47:54,572 : INFO : Finished loading new batch of 50000 documents
2017-03-30 02:47:57,298 : INFO : Loading new batch for index: 60000
2017-03-30 02:47:57,301 : INFO : Finished loading new batc